# Visualisierung der Kompetenzen-Excel-Tabelle

In [107]:
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
import pandas as pd

load_dotenv()
data = os.getenv('data')
sciebo = data[:-4]
file = sciebo + r"\DH-Ruhr\Recherche\Kompetenzen\DH_Kompetenzen_Übersicht_V2.2.xlsx"


In [108]:
df = pd.read_excel(file, index_col="id")

df.head()

c:\Users\Oliver\anaconda3\envs\py311\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning:

Data Validation extension is not supported and will be removed



,Status,Atomic Skills,tmp,Unterbereich,Bereich,Prio,Punkteanzahl BA,Punkteanzahl MA,Punkte gesamt,Punkte gewichtet,...,Zertifikat,Modul,Modul_Option2,Lernzieltyp,Lernziel_konkret,Ridsdale DL Framework Key Ability,Ridsdale DL Framework Competency,RUB_Angebot,UA_Ruhr_Angebot,Kommentare
id,,,,,,,,,,,,,,,,,,,,,
1,NaN,Computer-Probleme selber lösen (richtig google...,1,PC Grundlagen,Umgang mit dem PC,1,8,0,8,25,...,DH-Basic,Basismodul,Basismodul,Verstehen,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,"Dateien - Ordnerstruktur/-hierarchie, Pfade, B...",1,PC Grundlagen,Umgang mit dem PC,1,7,0,7,22,...,DH-Basic,Basismodul,Basismodul,Verstehen,NaN,Data Management,Data Organization,NaN,NaN,NaN
3,NaN,"Shell Grundlagen: Was ist die Shell?, Unix vs....",1,PC Grundlagen,Umgang mit dem PC,1,1,1,2,5,...,DH-Basic,Basismodul,Basismodul,Verstehen,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Tabellenkalkulation - Excel/Google Sheets,1,PC Grundlagen,Umgang mit dem PC,1,2,0,2,7,...,DH-Basic,Basismodul,Basismodul,Verstehen,NaN,Data Evaluation,Data Tools,NaN,NaN,NaN
5,NaN,Tastaturkürzel & Sonderzeichen,1,PC Grundlagen,Umgang mit dem PC,1,1,0,1,4,...,DH-Basic,Basismodul,Basismodul,Wissen,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
def brrrrrr(string, cutoff=20):
    brstring = ""
    while len(string) > cutoff:
        space_index = string[cutoff:].find(" ")
        if space_index == -1:
            break
        brstring += string[:cutoff + space_index].strip() + "<br>"
        string = string[cutoff + space_index:].strip()
    
    brstring += string
    return brstring
    
        

teststr = df['Atomic Skills'].iloc[92]
brrrrrr(teststr,20)

'Sonstige DH-Tools'

In [110]:

df["AS"] = df["Atomic Skills"].apply(lambda x: brrrrrr(x,15))
df["Prio_"] = df["Prio"].apply(lambda x: 4-x)

In [111]:
df.keys()

Index(['Status', 'Atomic Skills', 'tmp', 'Unterbereich', 'Bereich', 'Prio',
       'Punkteanzahl BA', 'Punkteanzahl MA', 'Punkte gesamt',
       'Punkte gewichtet', 'Zeitaufwand', 'Framework', 'Zertifikat', 'Modul',
       'Modul_Option2', 'Lernzieltyp', 'Lernziel_konkret',
       'Ridsdale DL Framework Key Ability', 'Ridsdale DL Framework Competency',
       'RUB_Angebot', 'UA_Ruhr_Angebot', 'Kommentare', 'AS', 'Prio_'],
      dtype='object')

In [112]:
# df[['Atomic Skills', 'Bereich', 'Unterbereich', 'Prio', 'Zeitaufwand','Lernzieltyp', 'Modul']].to_csv("dh_cert_skills_RUB.csv")

In [113]:
RUB_palette = {'Blau': '#17365c',
               'Grün': '#8dae10',
               'Grau': '#e7e7e7',
               'Gelb': '#ffcc00',
               'Orange': '#ed7102',
               'Rot': '#e6332a',
               'Dunkelrot': '#b61e3e',
               'Beige': '#c1baa3',
               'Dunkelgrün': '#8c8751',
               'Braun': '#9c5516',
               'Dunkelbraun': '#59201b'}


bereich_colors = {'DH Grundlagen': 'Grün',
                  'DH Methoden & Tools': 'Dunkelgrün',
                  'DH Projekt': 'Beige',
                  'Programmieren für DH': 'Dunkelrot',
                  'Statistik für DH': 'Blau',
                  'Umgang mit dem PC': 'Orange',
                  'Schlüsselkompetenzen Zertifikat "Digital Humanities"': 'Grau',}


df['color'] = df['Bereich'].apply(lambda x: RUB_palette[bereich_colors[x]])

RUB_colors = list(RUB_palette.values())

In [114]:
import plotly.io as pio
pio.templates

Templates configuration
-----------------------
    Default template: 'plotly'
    Available templates:
        ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none', 'draft', 'rub']

In [115]:
df['Modul'] = df.apply(lambda x: x.Modul if x.Bereich != 'Umgang mit dem PC' else 'Methodenmodul', axis=1)


# Ohne Module

In [116]:
import plotly.express as px

fig = px.treemap(df, path=[
    # px.Constant('Schlüsselkompetenzen Zertifikat "Digital Humanities"'),
    # 'Modul',
    'Bereich',
    'Unterbereich',
    'AS'
],
    values='Prio_',
    # hover_data=['Zeitaufwand'],
    # color=df['color'],
    # color_discrete_map={key: RUB_palette[bereich_colors[key]] for key in bereich_colors.keys()}
    # color_discrete_map={key: RUB_palette[bereich_colors[key]] for key in bereich_colors.keys()}
    color='Bereich',
    color_discrete_sequence=df['color'],
    color_discrete_map=dict(zip(df['Bereich'], df['color']))
)
fig.update_traces(marker=dict(cornerradius=5))
fig.update_layout(margin=dict(t=25, l=25, r=25, b=25))
fig.update_traces(hovertemplate='%{label}')
fig.update_layout(
    title={
    'text': 'Schlüsselkompetenzen Zertifikat "Digital Humanities"',  # Set your headline text here
    'y':0.98,  # Position the title slightly below the top edge
    'x':0.5,   # Center-align the title
    'xanchor': 'center',
    'yanchor': 'top'
    },
    # template='seaborn',
    font=dict(
        # family="Courier New, monospace",
        size=15,  # Set the font size here
        color="Black"
    )
)
fig.show()
fig.write_html("AS_Bereiche.html",  include_plotlyjs='cdn')
fig.write_image(f"img/AS_Bereiche.svg")

## Nach Workshop-Priorisierung

In [117]:
df.keys()

Index(['Status', 'Atomic Skills', 'tmp', 'Unterbereich', 'Bereich', 'Prio',
       'Punkteanzahl BA', 'Punkteanzahl MA', 'Punkte gesamt',
       'Punkte gewichtet', 'Zeitaufwand', 'Framework', 'Zertifikat', 'Modul',
       'Modul_Option2', 'Lernzieltyp', 'Lernziel_konkret',
       'Ridsdale DL Framework Key Ability', 'Ridsdale DL Framework Competency',
       'RUB_Angebot', 'UA_Ruhr_Angebot', 'Kommentare', 'AS', 'Prio_', 'color'],
      dtype='object')

In [120]:
import plotly.express as px

df['weight_hover'] = df.apply(lambda df: f"Abstimmungsergebnisse<br>B.A.: {df['Punkteanzahl BA']}<br>M.A.: {df['Punkteanzahl MA']}", axis=1)

fig = px.treemap(df, path=[
    # px.Constant('Schlüsselkompetenzen Zertifikat "Digital Humanities"'),
    # 'Modul',
    'Bereich',
    'Unterbereich',
    'AS'
],
    values='Punkte gewichtet',
    custom_data=['weight_hover'],
    # color=df['color'],
    # color_discrete_map={key: RUB_palette[bereich_colors[key]] for key in bereich_colors.keys()}
    # color_discrete_map={key: RUB_palette[bereich_colors[key]] for key in bereich_colors.keys()}
    color='Bereich',
    color_discrete_sequence=df['color'],
    color_discrete_map=dict(zip(df['Bereich'], df['color']))
)
fig.update_traces(marker=dict(cornerradius=5))
fig.update_layout(margin=dict(t=25, l=25, r=25, b=25))
fig.update_traces(hovertemplate='%{customdata[0]}<br><br>%{label}')
fig.update_layout(
    title={
    'text': 'Schlüsselkompetenzen Zertifikat "Digital Humanities"',  # Set your headline text here
    'y':0.98,  # Position the title slightly below the top edge
    'x':0.5,   # Center-align the title
    'xanchor': 'center',
    'yanchor': 'top'
    },
    # template='seaborn',
    font=dict(
        # family="Courier New, monospace",
        size=15,  # Set the font size here
        color="Black"
    )
)
fig.show()
fig.write_html("AS_Bereiche_gewichtet.html",  include_plotlyjs='cdn')
fig.write_image(f"img/AS_Bereiche_gewichtet.svg")

# Mit Modulen & AS

In [55]:
import plotly.express as px

fig = px.treemap(df, path=[
    px.Constant('Schlüsselkompetenzen Zertifikat "Digital Humanities"'),
    'Modul',
    'Bereich',
    'Unterbereich',
    'AS'
    ],
    values='Prio_',
    # hover_data=['Zeitaufwand'],
    # marker_colors = RUB_colors
    # color='Zertifikatsbereich'
)
fig.update_traces(marker=dict(cornerradius=5))
fig.update_layout(margin=dict(t=25, l=25, r=25, b=25))
fig.update_traces(hovertemplate='%{label}')
fig.update_layout(
    template='seaborn',
    font=dict(
        # family="Courier New, monospace",
        size=15,  # Set the font size here
        # color="RebeccaPurple"
    )
)
fig.show()
fig.write_html("AS_3Module.html",  include_plotlyjs='cdn')

# Mit Modulen ohne AS

In [56]:
import plotly.express as px

fig = px.treemap(df, path=[
    px.Constant('Schlüsselkompetenzen Zertifikat "Digital Humanities"'),
    'Modul',
    'Bereich',
    'Unterbereich',
    # 'AS'
    ],
    values='Prio_',
    # hover_data=['Zeitaufwand'],
    # marker_colors = RUB_colors
    # color='Zertifikatsbereich'
)
fig.update_traces(marker=dict(cornerradius=5))
fig.update_layout(margin=dict(t=25, l=25, r=25, b=25))
fig.update_traces(hovertemplate='%{label}')
fig.update_layout(
    template='seaborn',
    font=dict(
        # family="Courier New, monospace",
        size=15,  # Set the font size here
        # color="RebeccaPurple"
    )
)
fig.show()
fig.write_html("UB_3Module.html",  include_plotlyjs='cdn')

# 20-CP Variante

## Mit AS

In [57]:
import plotly.express as px

fig = px.treemap(df, path=[
    px.Constant('Schlüsselkompetenzen Zertifikat "Digital Humanities"'),
    'Modul_Option2',
    'Bereich',
    'Unterbereich',
    'AS'
    ],
    values='Prio_',
    # hover_data=['Zeitaufwand'],
    # marker_colors = RUB_colors
    # color='Zertifikatsbereich'
)
fig.update_traces(marker=dict(cornerradius=5))
fig.update_layout(margin=dict(t=25, l=25, r=25, b=25))
fig.update_traces(hovertemplate='%{label}')
fig.update_layout(
    template='seaborn',
    font=dict(
        # family="Courier New, monospace",
        size=15,  # Set the font size here
        # color="RebeccaPurple"
    )
)
fig.show()
fig.write_html("AS_4Module.html",  include_plotlyjs='cdn')

## Ohne AS

In [58]:
import plotly.express as px

fig = px.treemap(df, path=[
    px.Constant('Schlüsselkompetenzen Zertifikat "Digital Humanities"'),
    'Modul_Option2',
    'Bereich',
    'Unterbereich',
    # 'AS'
    ],
    values='Prio_',
    # hover_data=['Zeitaufwand'],
    # marker_colors = RUB_colors
    # color='Zertifikatsbereich'
)
fig.update_traces(marker=dict(cornerradius=5))
fig.update_layout(margin=dict(t=25, l=25, r=25, b=25))
fig.update_traces(hovertemplate='%{label}')
fig.show()
fig.write_html("UB_4Module.html",  include_plotlyjs='cdn')

In [59]:
import plotly.graph_objects as go
import plotly.io as pio

pio.templates["draft"] = go.layout.Template(
    layout_annotations=[
        dict(
            name="draft watermark",
            text="DRAFT",
            textangle=-30,
            opacity=0.1,
            font=dict(color="black", size=100),
            xref="paper",
            yref="paper",
            x=0.5,
            y=0.5,
            showarrow=False,
        )
    ]
)

In [60]:
import plotly.graph_objects as go
import plotly.io as pio

pio.templates["rub"] = go.layout.Template(
    layout_annotations=[
        dict(
            name="draft watermark",
            text="DRAFT",
            textangle=-30,
            opacity=0.1,
            font=dict(color="black", size=100),
            xref="paper",
            yref="paper",
            x=0.5,
            y=0.5,
            showarrow=False,
        )
    ]
)

# Poster Plots

In [61]:
import plotly.express as px

def plot_bereich(df, bereich):
    df = df[df['Bereich'] == bereich]
    fig = px.treemap(df, path=[
        px.Constant(bereich),
        'Unterbereich',
        'AS'
        ],
        # values='Prio_', # Skalierung nicht nötig für Plakate
        color_discrete_sequence =list(df['color'])
    )
    fig.update_traces(marker=dict(cornerradius=5))
    fig.update_layout(margin=dict(t=25, l=25, r=25, b=25))
    fig.update_traces(hovertemplate='%{label}')
    fig.update_layout(
        # template='seaborn',
        font=dict(
            # family="Courier New, monospace",
            size=15,  # Set the font size here
            color='black'
        )
    )
    fig.show()
    fig.write_image(f"img/{bereich.replace(' ', "_")}.svg")
    
print(RUB_palette)

for b in df.Bereich.value_counts().keys():
    # print(b)
    plot_bereich(df,b)

SyntaxError: f-string: unmatched '(' (3456754466.py, line 25)